# Analyzing fire department payrolls in California: 2011-2018

This notebook analyzes annual government payroll [data](https://publicpay.ca.gov/Reports/RawExport.aspx) compiled and released annually by the California state controller's office. The data include anonymized salary information for all employees at cities, counties, special districts and state government. 

---

### Load python tools

In [ ]:
import pandas as pd
import geopandas as gpd
import cpi
from urllib.request import urlopen 
import pyarrow
import jenkspy
import matplotlib.pyplot as plt
%matplotlib inline
import json
import numpy as np
from altair import datum
import altair as alt
alt.renderers.enable('notebook')
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.float_format = '{:,.0f}'.format

### Read the fire payroll data

In [ ]:
payroll_fire = pd.read_csv('/Users/mhustiles/data/data/controller/output/payroll_fire.csv')
agency_overtime = pd.read_csv('/Users/mhustiles/data/github/notebooks/payroll/output/agency_overtime.csv')

In [ ]:
len(payroll_fire)

---

### What about agencies in our area? 

In [ ]:
area_counties = ['LOS ANGELES', 'ORANGE', 'KERN', 'VENTURA', 'RIVERSIDE', 'SAN BERNARDINO']

In [ ]:
la_agency_overtime = agency_overtime[(agency_overtime['employercounty'] == 'LOS ANGELES') &\
                                         (agency_overtime['type'] != 'SPECIAL DISTRICT')]

la_area_agency_overtime = agency_overtime[(agency_overtime['employercounty'].isin(area_counties)) &\
                                         (agency_overtime['type'] != 'SPECIAL DISTRICT')]

In [ ]:
la_agency_grouped = agency_overtime[agency_overtime['employercounty'].isin(area_counties)]\
    .groupby(['year']).agg({'ot_share_wages':'mean'}).reset_index()

In [ ]:
la_agency_grouped

### Just cities

In [ ]:
cities_overtime = agency_overtime[(agency_overtime.type == 'CITY')]
cities_overtime.head()

### Statewide

In [ ]:
state = payroll_fire.groupby(['year']).agg({'adjusted_overtime':'sum', \
                                            'adjusted_overtime':'sum', \
                                            'adjusted_wages':'sum', \
                                            'adjusted_benefits':'sum'}).reset_index()

state.rename(columns={'adjusted_overtime': 'Overtime',\
                      'adjusted_wages': 'Wages',\
                      'adjusted_benefits':'Benefits',\
                     'year': 'Year'}, inplace=True)

In [ ]:
statemelt = pd.melt(state,id_vars=['Year'], value_vars=['Overtime','Wages','Benefits'],\
        var_name='Compensation', value_name='Total')

In [ ]:
alt.Chart(statemelt).mark_area().encode(
    x=alt.X("Year:N", title=' '),
    y=alt.Y("Total:Q", title=' ', axis=alt.Axis(tickCount=6, format='$,N')),
    color="Compensation:N"
).properties(width=500, title='Firefighter compensation in California')

--- 

### Charting firefighter wages and benefits in Los Angeles

In [ ]:
la = payroll_fire[(payroll_fire['employerfull'].str.contains('LOS_ANGELES'))]\
.groupby(['year', 'employerfull']).agg({'adjusted_overtime':'sum', \
                                            'adjusted_wages':'sum', \
                                            'adjusted_benefits':'sum'\
                                            }).reset_index()

### Clean up the column names

In [ ]:
la.rename(columns={'adjusted_overtime': 'Overtime',\
                      'adjusted_wages': 'Wages',\
                   'adjusted_benefits':'Benefits',\
                     'year': 'Year',
                      'employerfull': 'Place'}, inplace=True)

### Melt the dataframe for charting

In [ ]:
la_melt = pd.melt(la,id_vars=['Year', 'Place'], value_vars=['Overtime','Wages', 'Benefits'],\
        var_name='Compensation', value_name='Total')

In [ ]:
alt.Chart(la_melt).mark_area().encode(
    x=alt.X("Year:N", title=' '),
    y=alt.Y("Total:Q", title=' ', axis=alt.Axis(tickCount=6, format='$,N')),
    color="Compensation:N",
    facet='Place:N'
).properties(width=300, height=300, columns=2, title='Firefighter compensation in Los Angeles')

*Note: Los Angeles city [changed its methodology for reporting](https://www.dailynews.com/2019/06/14/los-angeles-finally-discloses-how-much-it-actually-pays-for-each-employees-benefits/) data in 2018, resulting in a spike.*

---

### Los Angeles-area fire agencies: Share of wages in overtime

In [ ]:
la_area = pd.DataFrame(la_agency_overtime[la_agency_overtime['employerfull'] \
                                          != 'LA_HABRA_HEIGHTS_CITY'][['employerfull', 'year', 'ot_share_wages']])

In [ ]:
alt.Chart(la_agency_grouped).mark_area().encode(
    x=alt.X('year:N', title='', axis=alt.Axis(format='')),
    y=alt.Y("ot_share_wages:Q",
        scale=alt.Scale(domain=(0, 25)), title="", axis=alt.Axis(format='', tickCount=6)),
).properties(height=500, width=500, columns=6, title='L.A. region: Overtime\'s share of firefighter wages')

In [ ]:
la_area['ot_share_wages'] = la_area['ot_share_wages']/100

In [ ]:
alt.Chart(la_area).mark_area().encode(
    x=alt.X('year:N', title='', axis=alt.Axis(format='')),
    y=alt.Y("ot_share_wages:Q", title="", axis=alt.Axis(format='%', tickCount=5)),
    facet='employerfull:N'
).properties(height=200, width=200, columns=4, \
             title='L.A.-area agencies: Overtime\'s share of firefighter wages')

### What's the rate of increase for OT, wages and benefits for each place? 

### What's the share of OT for the largest departments in the state? 